<a href="https://colab.research.google.com/github/Atreya-Tiger/NLP_L2/blob/clustering_based_aspect_category_detection/Notebooks/Case_Study_2_%7C_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install umap-learn hdbscan sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
  Created wheel for umap-

In [48]:
import re
import pandas as pd
import numpy as np
import ast
import umap
import hdbscan
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sentence_transformers import SentenceTransformer
import torch
import random
from functools import partial

In [6]:
df = pd.read_excel('/content/drive/MyDrive/02 Personal Study/05 NLP Case Study/train_data.xlsx')

In [7]:
df.head()

,Unnamed: 0.1,index,Unnamed: 0,sentence,aspects,sentiment,Aspect Category,unique_id
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,"['margarita', 'margarita', 'strawberry margari...","['Positive', 'Positive', 'Positive']","['Product', 'Product', 'Product']",0
1,1,5,"[50, 50]",I think Baja Bob is terrific ! I took another ...,"['mix', 'sugar']","['Neutral', 'Negative']","['Generic', 'Ingredient']",50
2,2,6,[1758],I gave the Pina Colada mix one star because it...,['citrus'],['Positive'],['Sensorial'],1758
3,3,8,"[371, 371, 371]",In principle a Splenda Margarita mix sounded l...,"['sugar', 'Splenda Margarita mix', 'drinks']","['Negative', 'Positive', 'Neutral']","['Ingredient', 'Product', 'Product Type']",371
4,4,9,[2488],You have to cut this down with Sprite Zero or ...,['water'],['Neutral'],['Generic'],2488


In [8]:
def explode_columns(df,col_list = []):
  if len(col_list)>0:
    for col in col_list:
      df[col] = df[col].apply(ast.literal_eval)

    df = df.explode(col_list)
  return df

In [9]:
# df['aspects'] = df['aspects'].apply(ast.literal_eval)
# df['Aspect Category'] = df['Aspect Category'].apply(ast.literal_eval)

In [10]:
df_exp = explode_columns(df.copy(),col_list = ['aspects','Aspect Category'])

In [11]:
df_exp.head()

,Unnamed: 0.1,index,Unnamed: 0,sentence,aspects,sentiment,Aspect Category,unique_id
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,strawberry margarita,"['Positive', 'Positive', 'Positive']",Product,0
1,1,5,"[50, 50]",I think Baja Bob is terrific ! I took another ...,mix,"['Neutral', 'Negative']",Generic,50
1,1,5,"[50, 50]",I think Baja Bob is terrific ! I took another ...,sugar,"['Neutral', 'Negative']",Ingredient,50


In [12]:
df_exp

,Unnamed: 0.1,index,Unnamed: 0,sentence,aspects,sentiment,Aspect Category,unique_id
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0
0,0,2,"[0, 0, 0]",This is as good as it gets for a 1 point marga...,strawberry margarita,"['Positive', 'Positive', 'Positive']",Product,0
1,1,5,"[50, 50]",I think Baja Bob is terrific ! I took another ...,mix,"['Neutral', 'Negative']",Generic,50
1,1,5,"[50, 50]",I think Baja Bob is terrific ! I took another ...,sugar,"['Neutral', 'Negative']",Ingredient,50
...,...,...,...,...,...,...,...,...
43030,43030,76178,[22772],"We love Moon Pies , however this box was stale...",box,['Negative'],Package,22772
43031,43031,76179,"[8778, 8778, 8778]",I ate Moon Pies growing up in Tennessee as a c...,taste,"['Positive', 'Positive', 'Neutral']",Sensorial,8778
43031,43031,76179,"[8778, 8778, 8778]",I ate Moon Pies growing up in Tennessee as a c...,corn syrup,"['Positive', 'Positive', 'Neutral']",Ingredient,8778
43031,43031,76179,"[8778, 8778, 8778]",I ate Moon Pies growing up in Tennessee as a c...,Moon Pies,"['Positive', 'Positive', 'Neutral']",Product,8778


In [13]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67598 entries, 0 to 43032
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0.1     67598 non-null  int64 
 1   index            67598 non-null  int64 
 2   Unnamed: 0       67598 non-null  object
 3   sentence         67598 non-null  object
 4   aspects          67598 non-null  object
 5   sentiment        67598 non-null  object
 6   Aspect Category  67598 non-null  object
 7   unique_id        67598 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 4.6+ MB


In [14]:
df_exp.columns

Index(['Unnamed: 0.1', 'index', 'Unnamed: 0', 'sentence', 'aspects',
       'sentiment', 'Aspect Category', 'unique_id'],
      dtype='object')

In [15]:
df_exp = df_exp[['sentence', 'aspects','sentiment', 'Aspect Category', 'unique_id']].copy()

In [16]:
df_exp.head()

,sentence,aspects,sentiment,Aspect Category,unique_id
0,This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0
0,This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0
0,This is as good as it gets for a 1 point marga...,strawberry margarita,"['Positive', 'Positive', 'Positive']",Product,0
1,I think Baja Bob is terrific ! I took another ...,mix,"['Neutral', 'Negative']",Generic,50
1,I think Baja Bob is terrific ! I took another ...,sugar,"['Neutral', 'Negative']",Ingredient,50


How many unique Aspect Categories are there?

In [17]:
df_exp['Aspect Category'].unique()

array(['Product', 'Generic', 'Ingredient', 'Sensorial', 'Product Type',
       'Format', 'Other', 'Package', 'Quality', 'Size', 'Nutrient',
       'Pricing', 'Shipping', 'Occasion'], dtype=object)

In [18]:
df_exp['Aspect Category'].nunique()

14

How many Unique Aspects are there?

In [19]:
df_exp['aspects'].nunique()

4845

In [20]:
def clean_text(text):
  text = re.sub('[^A-Za-z0-9 ]+', '', text)
  text = text.lower()
  text = text.strip()
  return text

In [21]:
# case for different entries may be different in case of aspects. Need to check
df_exp['aspects_cleaned'] = df_exp['aspects'].apply(clean_text)

In [22]:
df_exp['aspects_cleaned'].nunique()

3799

In [23]:
df_exp.shape

(67598, 6)

In [24]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67598 entries, 0 to 43032
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sentence         67598 non-null  object
 1   aspects          67598 non-null  object
 2   sentiment        67598 non-null  object
 3   Aspect Category  67598 non-null  object
 4   unique_id        67598 non-null  int64 
 5   aspects_cleaned  67598 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.6+ MB


How many aspects have multiple categories?

In [25]:
unique_cnt = df_exp.groupby('aspects_cleaned')['Aspect Category'].nunique()

In [26]:
unique_cnt

aspects_cleaned
1 cup serving coffee    1
1 ounce bags            1
1 oz snack              1
10 oz bag               1
10 packaging            1
                       ..
zip lock bag            1
zip lock bags           1
ziplock                 1
ziplocks                1
zoo animal crackers     1
Name: Aspect Category, Length: 3799, dtype: int64

In [27]:
df_test = pd.read_excel('/content/drive/MyDrive/02 Personal Study/05 NLP Case Study/test_data.xlsx')

In [28]:
df_test.head()

,Unnamed: 0.3,Unnamed: 0.2,index,Unnamed: 0.1,Unnamed: 0,sentence,aspects,sentiment,Aspect Category,unique_id
0,0,0,4,[5443],[1],I am not sure what the people expect who do no...,['margarita'],['Positive'],['Product'],1
1,1,1,26,[965],[8789],"Can always have a liquid mix on hand , so this...",['store'],['Positive'],['Other'],8789
2,2,2,31,[1307],[4189],"Waste of money , funky taste .",['taste'],['Negative'],['Sensorial'],4189
3,3,3,32,"[272, 273]","[4, 4]",Really did not like it at all . The mix is sup...,"['Margarita', 'mix']","['Neutral', 'Neutral']","['Product', 'Generic']",4
4,4,4,46,[3370],[10348],I first learned about this product on a cruise...,['product'],['Positive'],['Generic'],10348


In [29]:
df_test_exp = explode_columns(df_test.copy(),col_list = ['aspects','Aspect Category'])

In [30]:
df_test_exp.head()

,Unnamed: 0.3,Unnamed: 0.2,index,Unnamed: 0.1,Unnamed: 0,sentence,aspects,sentiment,Aspect Category,unique_id
0,0,0,4,[5443],[1],I am not sure what the people expect who do no...,margarita,['Positive'],Product,1
1,1,1,26,[965],[8789],"Can always have a liquid mix on hand , so this...",store,['Positive'],Other,8789
2,2,2,31,[1307],[4189],"Waste of money , funky taste .",taste,['Negative'],Sensorial,4189
3,3,3,32,"[272, 273]","[4, 4]",Really did not like it at all . The mix is sup...,Margarita,"['Neutral', 'Neutral']",Product,4
3,3,3,32,"[272, 273]","[4, 4]",Really did not like it at all . The mix is sup...,mix,"['Neutral', 'Neutral']",Generic,4


In [31]:
df_test_exp['aspects_cleaned'] = df_test_exp['aspects'].apply(clean_text)

In [32]:
df_test_exp['aspects_cleaned'].nunique()

1331

In [33]:
unique_cnt_test = df_test_exp.groupby('aspects_cleaned')['Aspect Category'].nunique()

In [34]:
unique_cnt_test

aspects_cleaned
11 oz bag                   1
12 bottles                  1
12 ounce bottle of syrup    1
12 oz glass bottle          1
12 pack                     1
                           ..
yogurt                      1
you amazon                  1
zip lock bags               1
ziploc bag                  1
ziplock bag                 1
Name: Aspect Category, Length: 1331, dtype: int64

In [35]:
aspect_not_in_train = set(unique_cnt_test.index) - set(unique_cnt.index)

In [36]:
len(aspect_not_in_train)

387

Clustering different aspects into required Categories

In [37]:
model =  SentenceTransformer('all-mpnet-base-v2')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [38]:
def embed(sentence,model):
  """
  wrapper function for generating message embeddings
  """
  embeddings = model.encode(sentence)

  return embeddings

In [43]:
embeddings_list = [embed(aspect,model = model) for aspect in list(unique_cnt.index)]

In [39]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components,
                      min_cluster_size,
                      random_state = None):
    """
    Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
    """

    umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors,
                                n_components=n_components,
                                metric='cosine',
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean',
                               cluster_selection_method='eom').fit(umap_embeddings)

    return clusters

In [40]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given cluster supplied from running hdbscan
    """

    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)

    return label_count, cost

In [41]:
def objective(params, embeddings, label_lower, label_upper):
    """
    Objective function for hyperopt to minimize, which incorporates constraints
    on the number of clusters we want to identify
    """

    clusters = generate_clusters(embeddings,
                                 n_neighbors = params['n_neighbors'],
                                 n_components = params['n_components'],
                                 min_cluster_size = params['min_cluster_size'],
                                 random_state = params['random_state'])

    label_count, cost = score_clusters(clusters, prob_threshold = 0.05)

    #15% penalty on the cost function if outside the desired range of groups
    if (label_count < label_lower) | (label_count > label_upper):
        penalty = 0.15
    else:
        penalty = 0

    loss = cost + penalty

    return {'loss': loss, 'label_count': label_count, 'status': STATUS_OK}

In [46]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,16)),
    "n_components": hp.choice('n_components', range(3,16)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,16)),
    "min_samples": None,
    "random_state": 42
}

label_lower = 30
label_upper = 100
max_evals = 100


In [42]:
def bayesian_search(embeddings, space, label_lower, label_upper, max_evals=100):
    """
    Perform bayseian search on hyperopt hyperparameter space to minimize objective function
    """

    trials = Trials()
    fmin_objective = partial(objective, embeddings=embeddings, label_lower=label_lower, label_upper=label_upper)
    best = fmin(fmin_objective,
                space = space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials=trials)

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"label count: {trials.best_trial['result']['label_count']}")

    best_clusters = generate_clusters(embeddings,
                                      n_neighbors = best_params['n_neighbors'],
                                      n_components = best_params['n_components'],
                                      min_cluster_size = best_params['min_cluster_size'],
                                      random_state = best_params['random_state'])

    return best_params, best_clusters, trials

In [50]:
best_params,best_clusters,trials = bayesian_search(embeddings_list, space=hspace,
                      label_lower=label_lower,
                      label_upper=label_upper,
                      max_evals=max_evals)

100%|██████████| 100/100 [49:53<00:00, 29.94s/trial, best loss: 0.17294024743353514]
best:
{'min_cluster_size': 12, 'min_samples': None, 'n_components': 12, 'n_neighbors': 4, 'random_state': 42}
label count: 80


In [51]:
best_params

{'min_cluster_size': 12,
 'min_samples': None,
 'n_components': 12,
 'n_neighbors': 4,
 'random_state': 42}

In [52]:
best_clusters

HDBSCAN(min_cluster_size=12)

In [53]:
df_exp.head()

,sentence,aspects,sentiment,Aspect Category,unique_id,aspects_cleaned
0,This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0,margarita
0,This is as good as it gets for a 1 point marga...,margarita,"['Positive', 'Positive', 'Positive']",Product,0,margarita
0,This is as good as it gets for a 1 point marga...,strawberry margarita,"['Positive', 'Positive', 'Positive']",Product,0,strawberry margarita
1,I think Baja Bob is terrific ! I took another ...,mix,"['Neutral', 'Negative']",Generic,50,mix
1,I think Baja Bob is terrific ! I took another ...,sugar,"['Neutral', 'Negative']",Ingredient,50,sugar


In [54]:
train_df = unique_cnt.reset_index()

In [55]:
train_df.head()

,aspects_cleaned,Aspect Category
0,1 cup serving coffee,1
1,1 ounce bags,1
2,1 oz snack,1
3,10 oz bag,1
4,10 packaging,1


In [56]:
train_df['embeddings'] = train_df['aspects_cleaned'].apply(lambda x :embed(x, model=model))

In [91]:
umap_embeddings = (umap.UMAP(n_neighbors=best_params['n_neighbors'],
                                n_components=best_params['n_components'],
                                metric='cosine',
                                random_state=best_params['random_state'])
                            .fit(train_df['embeddings'].tolist()))

clusterer = hdbscan.HDBSCAN(min_cluster_size = best_params['min_cluster_size'],
                               metric='euclidean', prediction_data=True,
                               cluster_selection_method='eom').fit(umap_embeddings.transform(train_df['embeddings'].tolist()))

In [63]:
clusterer

HDBSCAN(min_cluster_size=12, prediction_data=True)

In [92]:
def get_labels(best_params,clusterer,embeddings,umap_embeddings):
  reduced_embeddings = umap_embeddings.transform(embeddings)
  labels, membership_strengths = hdbscan.approximate_predict(clusterer, reduced_embeddings)

  return labels


In [94]:
labels = get_labels(best_params,clusterer,train_df['embeddings'].tolist(),umap_embeddings)

In [95]:
train_df['clusters'] = labels

In [96]:
train_df.head()

,aspects_cleaned,Aspect Category,embeddings,clusters
0,1 cup serving coffee,1,"[-0.0019861767, -0.057814505, 0.008084546, -0....",13
1,1 ounce bags,1,"[-0.021720072, -0.027055759, 0.0080201095, 0.0...",15
2,1 oz snack,1,"[0.02059984, -0.03900545, 0.024668878, -0.0422...",54
3,10 oz bag,1,"[-0.010282483, -0.018114718, 0.0032261405, -0....",15
4,10 packaging,1,"[0.00853375, -0.013654692, 0.008293894, 0.0128...",15


In [97]:
test_df = unique_cnt_test.reset_index()
test_df['embeddings'] = test_df['aspects_cleaned'].apply(lambda x :embed(x, model=model))

In [98]:
test_df['clusters'] = get_labels(best_params,clusterer,test_df['embeddings'].tolist(),umap_embeddings)

In [104]:
test_df.head()

,aspects_cleaned,Aspect Category,embeddings,clusters
0,11 oz bag,1,"[-0.018401925, -0.010895814, 0.010158479, -0.0...",15
1,12 bottles,1,"[0.03708248, 0.04720872, -0.015452163, -0.0041...",14
2,12 ounce bottle of syrup,1,"[0.040267885, 0.009543676, 0.02257713, -0.0142...",14
3,12 oz glass bottle,1,"[0.032168485, 0.012437751, 0.0012149984, -0.01...",14
4,12 pack,1,"[-0.012964742, 0.030354954, -0.01778375, -0.01...",15


In [99]:
test_df[test_df['clusters']==-1].shape

(220, 4)

In [100]:
test_df.shape

(1331, 4)

In [101]:
embed('10 oz bag',model=model).reshape(1,-1).shape

(1, 768)

In [105]:
label = get_labels(best_params,clusterer,embed('10 oz bag',model=model).reshape(1,-1),umap_embeddings)

In [106]:
label

array([15], dtype=int32)

In [121]:
aspect_to_category = {}

In [122]:
for idx,row in df_exp[['aspects_cleaned','Aspect Category']].drop_duplicates().iterrows():
  aspect_to_category[row['aspects_cleaned']] = row['Aspect Category']

In [123]:
train_df['Aspect Category'] = train_df['aspects_cleaned'].apply(lambda x :aspect_to_category[x])

In [124]:
train_df.head()

,aspects_cleaned,Aspect Category,embeddings,clusters,Predicted Category
0,1 cup serving coffee,Occasion,"[-0.0019861767, -0.057814505, 0.008084546, -0....",13,Product
1,1 ounce bags,Package,"[-0.021720072, -0.027055759, 0.0080201095, 0.0...",15,Package
2,1 oz snack,Product Type,"[0.02059984, -0.03900545, 0.024668878, -0.0422...",54,Product Type
3,10 oz bag,Package,"[-0.010282483, -0.018114718, 0.0032261405, -0....",15,Package
4,10 packaging,Package,"[0.00853375, -0.013654692, 0.008293894, 0.0128...",15,Package


In [147]:
cluster_to_cat_map = {}
for idx,row in  train_df.groupby('clusters')['Aspect Category'].agg(pd.Series.mode).reset_index().iterrows():
  if isinstance(row['Aspect Category'],str):
    cluster_to_cat_map[row['clusters']] = row['Aspect Category']
  else:
    cluster_to_cat_map[row['clusters']] = row['Aspect Category'][0]

In [148]:
train_df['Predicted Category'] = train_df['clusters'].apply(lambda x : cluster_to_cat_map[x])

In [149]:
aspect_to_category_test = {}
for idx,row in df_test_exp[['aspects_cleaned','Aspect Category']].drop_duplicates().iterrows():
  aspect_to_category_test[row['aspects_cleaned']] = row['Aspect Category']
test_df['Aspect Category'] = test_df['aspects_cleaned'].apply(lambda x :aspect_to_category_test[x])
test_df['Predicted Category'] = test_df['clusters'].apply(lambda x : cluster_to_cat_map[x])

In [150]:
train_df.head()

,aspects_cleaned,Aspect Category,embeddings,clusters,Predicted Category,Correct_Flag
0,1 cup serving coffee,Occasion,"[-0.0019861767, -0.057814505, 0.008084546, -0....",13,Product,NaN
1,1 ounce bags,Package,"[-0.021720072, -0.027055759, 0.0080201095, 0.0...",15,Package,NaN
2,1 oz snack,Product Type,"[0.02059984, -0.03900545, 0.024668878, -0.0422...",54,Product Type,NaN
3,10 oz bag,Package,"[-0.010282483, -0.018114718, 0.0032261405, -0....",15,Package,NaN
4,10 packaging,Package,"[0.00853375, -0.013654692, 0.008293894, 0.0128...",15,Package,NaN


In [151]:
train_df.shape

(3799, 6)

In [152]:
train_df['Correct_Flag'] = train_df[['Aspect Category','Predicted Category']].apply(lambda x: 1 if x['Aspect Category']==x['Predicted Category'] else 0, axis=1)

In [153]:
train_df['Correct_Flag'].sum()

2179

In [154]:
train_df[train_df['Correct_Flag']==1].shape

(2179, 6)

In [156]:
2179/3799

0.5735719926296394

In [157]:
test_df['Correct_Flag'] = test_df[['Aspect Category','Predicted Category']].apply(lambda x: 1 if x['Aspect Category']==x['Predicted Category'] else 0, axis=1)

In [158]:
test_df['Correct_Flag'].sum()

737

In [159]:
737/1331

0.5537190082644629

In [161]:
train_df.groupby('Aspect Category')['Correct_Flag'].sum()/train_df.groupby('Aspect Category')['Correct_Flag'].count()

Aspect Category
Format          0.174242
Generic         0.270833
Ingredient      0.480924
Nutrient        0.250000
Occasion        0.558442
Other           0.440000
Package         0.863874
Pricing         0.774194
Product         0.817130
Product Type    0.517857
Quality         0.000000
Sensorial       0.437086
Shipping        0.000000
Size            0.288136
Name: Correct_Flag, dtype: float64

In [162]:
test_df.groupby('Aspect Category')['Correct_Flag'].sum()/test_df.groupby('Aspect Category')['Correct_Flag'].count()

Aspect Category
Format          0.181818
Generic         0.224490
Ingredient      0.498471
Nutrient        0.200000
Occasion        0.791667
Other           0.582090
Package         0.800000
Pricing         0.812500
Product         0.784722
Product Type    0.629630
Quality         0.000000
Sensorial       0.391026
Shipping        0.000000
Size            0.176471
Name: Correct_Flag, dtype: float64

In [166]:
cols_to_keep = ['aspects_cleaned','Aspect Category','clusters','Predicted Category','Correct_Flag']

In [169]:
train_df[cols_to_keep].to_excel('/content/drive/MyDrive/02 Personal Study/05 NLP Case Study/train_df.xlsx',index=False)
test_df[cols_to_keep].to_excel('/content/drive/MyDrive/02 Personal Study/05 NLP Case Study/test_df.xlsx',index=False)